In [1]:
import requests
import json
import prettytable
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 
import io

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
file = "data_files/national_M2018_dl.xlsx"
bls_msa_may2018_df = pd.read_excel(file)
bls_msa_may2018_df.head()

,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,00-0000,All Occupations,total,144733270,0.1,24.98,51960,0.1,9.95,12.37,18.58,30.06,47.31,20690,25740,38640,62510,98410,NaN,NaN
1,11-0000,Management Occupations,major,7616650,0.2,58.44,121560,0.1,23.68,34.08,50.11,72.93,#,49260,70880,104240,151700,#,NaN,NaN
2,11-1000,Top Executives,minor,2535640,0.3,61.66,128240,0.2,20.86,31.73,49.73,78.81,#,43400,66000,103450,163930,#,NaN,NaN
3,11-1010,Chief Executives,broad,195530,0.7,96.22,200140,0.5,32.86,55.75,91.15,#,#,68360,115960,189600,#,#,NaN,NaN
4,11-1011,Chief Executives,detailed,195530,0.7,96.22,200140,0.5,32.86,55.75,91.15,#,#,68360,115960,189600,#,#,NaN,NaN


In [4]:
bls_msa_may2018_df.nunique()

OCC_CODE     1373
OCC_TITLE    1102
OCC_GROUP       5
TOT_EMP      1049
EMP_PRSE      126
H_MEAN        865
A_MEAN        987
MEAN_PRSE      54
H_PCT10       735
H_PCT25       808
H_MEDIAN      853
H_PCT75       881
H_PCT90       870
A_PCT10       917
A_PCT25       940
A_MEDIAN      988
A_PCT75       989
A_PCT90       975
ANNUAL          1
HOURLY          1
dtype: int64

In [5]:
bls_msa_may2018_df.columns

Index(['OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'EMP_PRSE', 'H_MEAN',
       'A_MEAN', 'MEAN_PRSE', 'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75',
       'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90',
       'ANNUAL', 'HOURLY'],
      dtype='object')

In [6]:
# bls_nat_df = bls_df.drop(['JOBS_1000',
#                           'LOC_QUOTIENT',
#                           'PCT_TOTAL',
#                           'ANNUAL',
#                           'HOURLY',
#                           'AREA',
#                           'AREA_TITLE',
#                           'AREA_TYPE' , 
#                           'PRIM_STATE',
#                           'NAICS',
#                           'NAICS_TITLE',
#                           'I_GROUP',
#                           'OWN_CODE',
#                           'H_PCT90',
#                           'H_PCT10',
#                           'H_PCT25',
#                           'H_MEDIAN',
#                           'H_PCT75',
#                           'A_PCT10',
#                           'A_PCT25',
#                           'A_MEDIAN',
#                           'A_PCT75',
#                           'A_PCT90'] ,axis=1)
# bls_nat_df.head()

In [7]:
list_nat = bls_msa_may2018_df.columns.to_list()
list_nat

['OCC_CODE',
 'OCC_TITLE',
 'OCC_GROUP',
 'TOT_EMP',
 'EMP_PRSE',
 'H_MEAN',
 'A_MEAN',
 'MEAN_PRSE',
 'H_PCT10',
 'H_PCT25',
 'H_MEDIAN',
 'H_PCT75',
 'H_PCT90',
 'A_PCT10',
 'A_PCT25',
 'A_MEDIAN',
 'A_PCT75',
 'A_PCT90',
 'ANNUAL',
 'HOURLY']

In [8]:
list_nat = [col.lower() for col in list_nat]
list_nat

['occ_code',
 'occ_title',
 'occ_group',
 'tot_emp',
 'emp_prse',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90',
 'annual',
 'hourly']

In [9]:
bls_msa_may2018_df.columns = list_nat

In [10]:
cols = ['tot_emp',
 'emp_prse',
 'h_mean',
 'a_mean',
 'mean_prse',
 ]

for col in cols:
    print(col)
    bls_msa_may2018_df[col] = pd.to_numeric(bls_msa_may2018_df[col], errors='coerce')

bls_msa_may2018_df.head()

tot_emp
emp_prse
h_mean
a_mean
mean_prse


,occ_code,occ_title,occ_group,tot_emp,emp_prse,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,annual,hourly
0,00-0000,All Occupations,total,144733270,0.1,24.98,51960.0,0.1,9.95,12.37,18.58,30.06,47.31,20690,25740,38640,62510,98410,NaN,NaN
1,11-0000,Management Occupations,major,7616650,0.2,58.44,121560.0,0.1,23.68,34.08,50.11,72.93,#,49260,70880,104240,151700,#,NaN,NaN
2,11-1000,Top Executives,minor,2535640,0.3,61.66,128240.0,0.2,20.86,31.73,49.73,78.81,#,43400,66000,103450,163930,#,NaN,NaN
3,11-1010,Chief Executives,broad,195530,0.7,96.22,200140.0,0.5,32.86,55.75,91.15,#,#,68360,115960,189600,#,#,NaN,NaN
4,11-1011,Chief Executives,detailed,195530,0.7,96.22,200140.0,0.5,32.86,55.75,91.15,#,#,68360,115960,189600,#,#,NaN,NaN


In [11]:
bls_msa_may2018_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1379 entries, 0 to 1378
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   occ_code   1379 non-null   object 
 1   occ_title  1379 non-null   object 
 2   occ_group  1379 non-null   object 
 3   tot_emp    1379 non-null   int64  
 4   emp_prse   1379 non-null   float64
 5   h_mean     1297 non-null   float64
 6   a_mean     1373 non-null   float64
 7   mean_prse  1379 non-null   float64
 8   h_pct10    1379 non-null   object 
 9   h_pct25    1379 non-null   object 
 10  h_median   1379 non-null   object 
 11  h_pct75    1379 non-null   object 
 12  h_pct90    1379 non-null   object 
 13  a_pct10    1379 non-null   object 
 14  a_pct25    1379 non-null   object 
 15  a_median   1379 non-null   object 
 16  a_pct75    1379 non-null   object 
 17  a_pct90    1379 non-null   object 
 18  annual     82 non-null     object 
 19  hourly     6 non-null      object 
dtypes: float

In [12]:
bls_msa_may2018_df = bls_msa_may2018_df.dropna()

In [13]:
bls_msa_may2018_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   occ_code   0 non-null      object 
 1   occ_title  0 non-null      object 
 2   occ_group  0 non-null      object 
 3   tot_emp    0 non-null      int64  
 4   emp_prse   0 non-null      float64
 5   h_mean     0 non-null      float64
 6   a_mean     0 non-null      float64
 7   mean_prse  0 non-null      float64
 8   h_pct10    0 non-null      object 
 9   h_pct25    0 non-null      object 
 10  h_median   0 non-null      object 
 11  h_pct75    0 non-null      object 
 12  h_pct90    0 non-null      object 
 13  a_pct10    0 non-null      object 
 14  a_pct25    0 non-null      object 
 15  a_median   0 non-null      object 
 16  a_pct75    0 non-null      object 
 17  a_pct90    0 non-null      object 
 18  annual     0 non-null      object 
 19  hourly     0 non-null      object 
dtypes: float64(4), int64(1

In [28]:
file_metro = "data_files\MSA_M2018_dl.xlsx"
bls_metro_may2018_df = pd.read_excel(file_metro)
bls_metro_may2018_df.head()

,PRIM_STATE,AREA,AREA_NAME,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,JOBS_1000,LOC QUOTIENT,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,171770,1.1,1000,1,28.57,59430,2.1,11.41,14.94,22.68,35.82,51.21,23740,31070,47170,74510,106510,NaN,NaN
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10610,2.5,61.779,1.17,55.59,115630,1.8,26.68,36.03,48.94,64.7,93.24,55490,74950,101800,134580,193930,NaN,NaN
2,AK,11260,"Anchorage, AK",11-1011,Chief Executives,detailed,400,8.7,2.307,1.71,89,185120,4.4,41.22,57.53,78.04,#,#,85730,119660,162320,#,#,NaN,NaN
3,AK,11260,"Anchorage, AK",11-1021,General and Operations Managers,detailed,3630,4.2,21.157,1.34,59.23,123200,2.7,26.53,35.99,50.1,71.35,#,55190,74860,104210,148400,#,NaN,NaN
4,AK,11260,"Anchorage, AK",11-1031,Legislators,detailed,50,1.3,0.293,0.84,*,41580,15,*,*,*,*,*,20690,27640,51130,51140,51140,True,NaN


In [15]:
bls_metro_may2018_df.nunique()

PRIM_STATE         52
AREA              395
AREA_NAME         395
OCC_CODE          825
OCC_TITLE         825
OCC_GROUP           3
TOT_EMP          3852
EMP_PRSE          501
JOBS_1000       21684
LOC QUOTIENT     1553
H_MEAN           7881
A_MEAN          13846
MEAN_PRSE         297
H_PCT10          4843
H_PCT25          5799
H_MEDIAN         6701
H_PCT75          7760
H_PCT90          8708
A_PCT10          8490
A_PCT25         10225
A_MEDIAN        12239
A_PCT75         14635
A_PCT90         16796
ANNUAL              1
HOURLY              1
dtype: int64

In [16]:
# bls_metro_may2018_df = bls_metro_may2018_df.drop(['AREA_TYPE',
#                                     'NAICS',
#                                     'NAICS_TITLE',
#                                     'I_GROUP',
#                                     'OWN_CODE',
#                                     'PCT_TOTAL',
#                                     'ANNUAL',
#                                     'HOURLY'] ,axis=1)
# bls_metro_may2018_df.head()

In [17]:
x = bls_metro_may2018_df.columns.to_list()
x

['PRIM_STATE',
 'AREA',
 'AREA_NAME',
 'OCC_CODE',
 'OCC_TITLE',
 'OCC_GROUP',
 'TOT_EMP',
 'EMP_PRSE',
 'JOBS_1000',
 'LOC QUOTIENT',
 'H_MEAN',
 'A_MEAN',
 'MEAN_PRSE',
 'H_PCT10',
 'H_PCT25',
 'H_MEDIAN',
 'H_PCT75',
 'H_PCT90',
 'A_PCT10',
 'A_PCT25',
 'A_MEDIAN',
 'A_PCT75',
 'A_PCT90',
 'ANNUAL',
 'HOURLY']

In [18]:
x = [col.lower() for col in x]
x

['prim_state',
 'area',
 'area_name',
 'occ_code',
 'occ_title',
 'occ_group',
 'tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc quotient',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90',
 'annual',
 'hourly']

In [19]:
bls_metro_may2018_df.columns = x

In [20]:
bls_metro_may2018_df.head()

,prim_state,area,area_name,occ_code,occ_title,occ_group,tot_emp,emp_prse,jobs_1000,loc quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,annual,hourly
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,171770,1.1,1000,1,28.57,59430,2.1,11.41,14.94,22.68,35.82,51.21,23740,31070,47170,74510,106510,NaN,NaN
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10610,2.5,61.779,1.17,55.59,115630,1.8,26.68,36.03,48.94,64.7,93.24,55490,74950,101800,134580,193930,NaN,NaN
2,AK,11260,"Anchorage, AK",11-1011,Chief Executives,detailed,400,8.7,2.307,1.71,89,185120,4.4,41.22,57.53,78.04,#,#,85730,119660,162320,#,#,NaN,NaN
3,AK,11260,"Anchorage, AK",11-1021,General and Operations Managers,detailed,3630,4.2,21.157,1.34,59.23,123200,2.7,26.53,35.99,50.1,71.35,#,55190,74860,104210,148400,#,NaN,NaN
4,AK,11260,"Anchorage, AK",11-1031,Legislators,detailed,50,1.3,0.293,0.84,*,41580,15,*,*,*,*,*,20690,27640,51130,51140,51140,True,NaN


In [21]:
x

['prim_state',
 'area',
 'area_name',
 'occ_code',
 'occ_title',
 'occ_group',
 'tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc quotient',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90',
 'annual',
 'hourly']

In [22]:
cols = ['tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc quotient',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90']

for col in cols:
    bls_metro_may2018_df[col] = pd.to_numeric(bls_metro_may2018_df[col], errors='coerce')

bls_metro_may2018_df.head()

,prim_state,area,area_name,occ_code,occ_title,occ_group,tot_emp,emp_prse,jobs_1000,loc quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,annual,hourly
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,171770.0,1.1,1000.000,1.00,28.57,59430.0,2.1,11.41,14.94,22.68,35.82,51.21,23740.0,31070.0,47170.0,74510.0,106510.0,NaN,NaN
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10610.0,2.5,61.779,1.17,55.59,115630.0,1.8,26.68,36.03,48.94,64.70,93.24,55490.0,74950.0,101800.0,134580.0,193930.0,NaN,NaN
2,AK,11260,"Anchorage, AK",11-1011,Chief Executives,detailed,400.0,8.7,2.307,1.71,89.00,185120.0,4.4,41.22,57.53,78.04,NaN,NaN,85730.0,119660.0,162320.0,NaN,NaN,NaN,NaN
3,AK,11260,"Anchorage, AK",11-1021,General and Operations Managers,detailed,3630.0,4.2,21.157,1.34,59.23,123200.0,2.7,26.53,35.99,50.10,71.35,NaN,55190.0,74860.0,104210.0,148400.0,NaN,NaN,NaN
4,AK,11260,"Anchorage, AK",11-1031,Legislators,detailed,50.0,1.3,0.293,0.84,NaN,41580.0,15.0,NaN,NaN,NaN,NaN,NaN,20690.0,27640.0,51130.0,51140.0,51140.0,True,NaN


In [23]:
bls_metro_may2018_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143905 entries, 0 to 143904
Data columns (total 25 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   prim_state    143905 non-null  object 
 1   area          143905 non-null  int64  
 2   area_name     143905 non-null  object 
 3   occ_code      143905 non-null  object 
 4   occ_title     143905 non-null  object 
 5   occ_group     143905 non-null  object 
 6   tot_emp       131231 non-null  float64
 7   emp_prse      131231 non-null  float64
 8   jobs_1000     131231 non-null  float64
 9   loc quotient  131231 non-null  float64
 10  h_mean        134672 non-null  float64
 11  a_mean        141290 non-null  float64
 12  mean_prse     141076 non-null  float64
 13  h_pct10       134672 non-null  float64
 14  h_pct25       134470 non-null  float64
 15  h_median      133916 non-null  float64
 16  h_pct75       133074 non-null  float64
 17  h_pct90       131100 non-null  float64
 18  a_pc

In [24]:
bls_metro_may2018_df = bls_metro_may2018_df.dropna()

In [25]:
bls_metro_may2018_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   prim_state    0 non-null      object 
 1   area          0 non-null      int64  
 2   area_name     0 non-null      object 
 3   occ_code      0 non-null      object 
 4   occ_title     0 non-null      object 
 5   occ_group     0 non-null      object 
 6   tot_emp       0 non-null      float64
 7   emp_prse      0 non-null      float64
 8   jobs_1000     0 non-null      float64
 9   loc quotient  0 non-null      float64
 10  h_mean        0 non-null      float64
 11  a_mean        0 non-null      float64
 12  mean_prse     0 non-null      float64
 13  h_pct10       0 non-null      float64
 14  h_pct25       0 non-null      float64
 15  h_median      0 non-null      float64
 16  h_pct75       0 non-null      float64
 17  h_pct90       0 non-null      float64
 18  a_pct10       0 non-null      float64
 19 

In [29]:
bls_metro_may2018_df.head(5)

,PRIM_STATE,AREA,AREA_NAME,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,JOBS_1000,LOC QUOTIENT,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,171770,1.1,1000,1,28.57,59430,2.1,11.41,14.94,22.68,35.82,51.21,23740,31070,47170,74510,106510,NaN,NaN
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10610,2.5,61.779,1.17,55.59,115630,1.8,26.68,36.03,48.94,64.7,93.24,55490,74950,101800,134580,193930,NaN,NaN
2,AK,11260,"Anchorage, AK",11-1011,Chief Executives,detailed,400,8.7,2.307,1.71,89,185120,4.4,41.22,57.53,78.04,#,#,85730,119660,162320,#,#,NaN,NaN
3,AK,11260,"Anchorage, AK",11-1021,General and Operations Managers,detailed,3630,4.2,21.157,1.34,59.23,123200,2.7,26.53,35.99,50.1,71.35,#,55190,74860,104210,148400,#,NaN,NaN
4,AK,11260,"Anchorage, AK",11-1031,Legislators,detailed,50,1.3,0.293,0.84,*,41580,15,*,*,*,*,*,20690,27640,51130,51140,51140,True,NaN


In [30]:
bls_metro_may2018_df["Date"]="May 2018"

In [31]:
bls_metro_may2018_df.head(5)

,PRIM_STATE,AREA,AREA_NAME,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,JOBS_1000,LOC QUOTIENT,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY,Date
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,171770,1.1,1000,1,28.57,59430,2.1,11.41,14.94,22.68,35.82,51.21,23740,31070,47170,74510,106510,NaN,NaN,May 2018
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10610,2.5,61.779,1.17,55.59,115630,1.8,26.68,36.03,48.94,64.7,93.24,55490,74950,101800,134580,193930,NaN,NaN,May 2018
2,AK,11260,"Anchorage, AK",11-1011,Chief Executives,detailed,400,8.7,2.307,1.71,89,185120,4.4,41.22,57.53,78.04,#,#,85730,119660,162320,#,#,NaN,NaN,May 2018
3,AK,11260,"Anchorage, AK",11-1021,General and Operations Managers,detailed,3630,4.2,21.157,1.34,59.23,123200,2.7,26.53,35.99,50.1,71.35,#,55190,74860,104210,148400,#,NaN,NaN,May 2018
4,AK,11260,"Anchorage, AK",11-1031,Legislators,detailed,50,1.3,0.293,0.84,*,41580,15,*,*,*,*,*,20690,27640,51130,51140,51140,True,NaN,May 2018


In [32]:
#create CSV files to be used to load database as needed
bls_metro_may2018_df.to_csv('data_files/bls_metro_may2018_clean.csv')
# bls_nat_df.to_csv('data_files/bls_nat_clean.csv')